# **Pokémon Diffusion<a id="top"></a>**

> #### ``05-DDPM-Final_Model.ipynb``

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 14/07/2024</small></i></div>

---

**Objetivo:** 
Este notebook tiene como objetivo mostrar los resultados obtenidos tras el entrenamiento del modelo DDPM para la generación de imágenes de Pokémon condicionadas al tipo de Pokémon.

- [0. Imports](#-0.-Imports)

- [1. Carga del modelo entrenado](#1-caga-del-modelo-entrenado)
- [2. Resultados del modelo](#2-resultados-del-modelo)

---

# 0. Imports

Una vez introducido el objetivo de este notebook, se importan las librerías necesarias para el desarrollo del apartado.

In [ ]:
# Import necessary libraries
# =====================================================================
import tensorflow as tf
import configparser
from src.model.build_model import build_unet
from src.utils import *
from src.utils.config import parse_config
from src.model.diffusion_funcionality import *

In [ ]:
# Use the GPU
# =====================================================================
gpus_list = tf.config.list_physical_devices("GPU")
gpu = gpus_list[0]
tf.config.experimental.set_memory_growth(gpu, True)

print("GPUs Available: ", gpus_list)

In [ ]:
# Set config file
# =====================================================================
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

hyperparameters = parse_config(config, "hyperparameters")

IMG_SIZE = hyperparameters["img_size"]
NUM_CLASSES = hyperparameters["num_classes"]
BATCH_SIZE = hyperparameters["batch_size"]
EPOCHS = hyperparameters["epochs"]

TIMESTEPS = hyperparameters["timesteps"]
SCHEDULER = hyperparameters["scheduler"]
BETA_START = hyperparameters["beta_start"]
BETA_END = hyperparameters["beta_end"]
S = hyperparameters["s"]

## 1. Caga del modelo entrenado

Cargamos el modelo entrenado en el notebook anterior. Para ello, se selecciona el path del modelo y se carga, gracias a la función implementada en `diffusion_functionality.py`.

In [ ]:
# Load the model
# =====================================================================
load_path = f"{MODELS_PATH}/final_diffusion_{IMG_SIZE}x{IMG_SIZE}_batch{BATCH_SIZE}_epochs{EPOCHS}.weights.h5"
u_net = build_unet(IMG_SIZE, NUM_CLASSES, dropout_rate=0.1)
ema_u_net = build_unet(IMG_SIZE, NUM_CLASSES, dropout_rate=0.1) # EMA model for smoother training
ema_u_net.set_weights(u_net.get_weights()) # Initialize EMA model with the same weights


model_loaded = DiffusionModel.load_model(
    load_path,
    u_net,
    ema_u_net,
    IMG_SIZE,
    NUM_CLASSES,
    TIMESTEPS,
    BETA_START,
    BETA_END,
    S,
    SCHEDULER,
    ema=0.999
)

In [ ]:
model_loaded.plot_samples(1) # test, borrar luego

## 2. Resultados del modelo

Tras cargar el modelo, se generan imágenes de Pokémon condicionadas a su tipo. Para ello, se seleccionan los tipos de Pokémon (opcional) y se generan las imágenes correspondientes.

In [ ]:
# Sample some sprites
# =====================================================================
model_loaded.plot_samples(6)
model_loaded.plot_samples(6)
model_loaded.plot_samples(6)
model_loaded.plot_samples(6)
model_loaded.plot_samples(6)
model_loaded.plot_samples(6)

In [ ]:
model_loaded.plot_samples(6, "Bug")
model_loaded.plot_samples(6, "Dark")
model_loaded.plot_samples(6, "Dragon")
model_loaded.plot_samples(6, "Electric")
model_loaded.plot_samples(6, "Fairy")
model_loaded.plot_samples(6, "Fighting")

In [ ]:
model_loaded.plot_samples(6, "Fire")
model_loaded.plot_samples(6, "Flying")
model_loaded.plot_samples(6, "Ghost")
model_loaded.plot_samples(6, "Grass")
model_loaded.plot_samples(6, "Ground")
model_loaded.plot_samples(6, "Ice")

In [ ]:
model_loaded.plot_samples(6, "Normal")
model_loaded.plot_samples(6, "Poison")
model_loaded.plot_samples(6, "Psychic")
model_loaded.plot_samples(6, "Rock")
model_loaded.plot_samples(6, "Steel")
model_loaded.plot_samples(6, "Water")